# Variable Creation

In this notebook the different measures which are computed on investor (subsidiary) level will be created and transformed to firm level.

#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import math
import matplotlib.pyplot as plt
import seaborn as sns

#### Data Directory Locations

In [2]:
backup_dir = '00 Backup\\'
investor_data_dir = '01 HoldingsData\\'
subsidiary_data_dir = '02 Subsidiary Data\\'
false_positives_dir = '03 False Positives\\'
output_dir = '04 Output\\'
adj_dir = '05 Adjacency Matrix\\'
graph_output_dir = '06 Investor Network Graphs\\'
acc_dir = '07 Accounting Data\\'

#### Constant Variables

In [3]:
STARTYEAR = 2006
ENDYEAR = 2020
YEARS = np.arange(STARTYEAR, ENDYEAR + 1)
#YEARS = [2007]

## Data Cleaning Process

In [4]:
a = pd.read_stata(f'{acc_dir}Full_Accounting_Data.dta')

In [5]:
df = a


df['RESEARCHDEVELOPMENT'] = df['RESEARCHDEVELOPMENT'].fillna(0)
# df['TOTALINTANGIBLEOTASSETSNET'] = df['TOTALINTANGIBLEOTASSETSNET'].fillna(0)

df = df[(df[['ISIN','NETSALESORREVENUES','OPERATINGINCOME', 'TOTALASSETS', 'TOTALDEBT', 'TOTALLIABILITIES', 'MARKETCAPITALIZATION',
             'CAPITALEXPENDITURES','NETINCOMEBASIC']].notna()).all(axis=1)]
df = df[(df[['ISIN','NETSALESORREVENUES','OPERATINGINCOME', 'TOTALASSETS', 'TOTALDEBT', 'TOTALLIABILITIES', 'MARKETCAPITALIZATION',
             'CAPITALEXPENDITURES','NETINCOMEBASIC']] != 'n/a').all(axis=1)]
df = df[(df[['ISIN','NETSALESORREVENUES','OPERATINGINCOME', 'TOTALASSETS', 'TOTALDEBT', 'TOTALLIABILITIES', 'MARKETCAPITALIZATION',
             'CAPITALEXPENDITURES','NETINCOMEBASIC']] != 'na').all(axis=1)]


df = df[(df[['NETSALESORREVENUES', 'CAPITALEXPENDITURES', 'TOTALINTANGIBLEOTASSETSNET', 'RESEARCHDEVELOPMENT', 'TOTALASSETS', 'TOTALDEBT',
             'TOTALLIABILITIES', 'MARKETCAPITALIZATION']] >= 0).all(axis=1)]



ACC_VARIABLES = [['CAPITALEXPENDITURES', 'COMMONSHAREHOLDERSEQUITY', 'MARKETCAPITALIZATION', 'NETINCOMEBASIC', 'NETSALESORREVENUES', 'OPERATINGINCOME',
                'TOTALASSETS', 'TOTALDEBT', 'TOTALLIABILITIES', 'TOTALINTANGIBLEOTASSETSNET', 'RESEARCHDEVELOPMENT', 'CURRENTASSETSTOTAL',
                  'PROPERTYPLANTEQUIPNET', 'SELLINGGENERALADMINISTRAT']]

for i in ACC_VARIABLES:    
    df[i] = df[i] * 1000



df = df[df['ISIN'] != 'n/a']
df = df[df['TYPE'] == 'EQ']
df = df[df['ISINID'] == 'P']
df = df[df['MAJOR'] == 'Y']
df = df[df['GEOGN'] == df['GEOLN']]

df = df[df['TOTALASSETS'] > 0]
df = df[df['TOTALLIABILITIES'] > 0]
df = df[df['NETSALESORREVENUES'] > 0]

df['TOTALEQUITY'] = df['TOTALASSETS'] - df['TOTALLIABILITIES']
df = df[df['TOTALEQUITY'] > 0]



df = df[df.year > 2004]

df = df[df['WC07021'] != 'na']
df = df[df['WC07021'] != 'n/a']
df['WC07021'] = df['WC07021'].astype(int)


### Standard Industry Division: https://siccode.com/
df['Industry_Div'] = ''
df.loc[(df['WC07021'] >= 100) & (df['WC07021'] <= 999), 'Industry_Div'] = '[1] Agriculture, Forestry, Fishing'
df.loc[(df['WC07021'] >= 1000) & (df['WC07021'] <= 1799), 'Industry_Div'] = '[2] Mining and Construction'
df.loc[(df['WC07021'] >= 2000) & (df['WC07021'] <= 2999), 'Industry_Div'] = '[3] Light Manufacturing'
df.loc[(df['WC07021'] >= 3000) & (df['WC07021'] <= 3999), 'Industry_Div'] = '[4] Heavy Manufacturing'
df.loc[(df['WC07021'] >= 4000) & (df['WC07021'] <= 4999), 'Industry_Div'] = '[5] Transportation and Public Utility'
df.loc[(df['WC07021'] >= 5000) & (df['WC07021'] <= 5999), 'Industry_Div'] = '[6] Wholesale and Retail'
df.loc[(df['WC07021'] >= 6000) & (df['WC07021'] <= 6999), 'Industry_Div'] = '[7] Finance, Insurance, Real Estate'
df.loc[(df['WC07021'] >= 7000) & (df['WC07021'] <= 8999), 'Industry_Div'] = '[8] Services'
df.loc[(df['WC07021'] >= 9100) & (df['WC07021'] <= 9729), 'Industry_Div'] = '[9] Public Administration'
df.loc[(df['WC07021'] >= 9900) & (df['WC07021'] <= 9999), 'Industry_Div'] = 'Nonclassifiable'


### FAMA FRENCH: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_17_ind_port.html
df['Industry_FF'] = ''

df.loc[(df['WC07021'].between(100, 299)) |
       (df['WC07021'].between(700, 799)) |
       (df['WC07021'].between(900, 999)) |
       (df['WC07021'].between(2000, 2099)) |
       (df['WC07021'].between(5140, 5159)) |
       (df['WC07021'].between(5180, 5182)) |
       (df['WC07021'].between(5191, 5191)),
       'Industry_FF'] = '[1] Food'

df.loc[(df['WC07021'].between(1000, 1099)) |
       (df['WC07021'].between(1200, 1299)) |
       (df['WC07021'].between(1400, 1499)) |
       (df['WC07021'].between(5050, 5052)),
       'Industry_FF'] = '[2] Mining and Minerals'

df.loc[(df['WC07021'].between(1300, 1389)) |
       (df['WC07021'].between(2900, 2912)) |
       (df['WC07021'].between(1400, 1499)) |
       (df['WC07021'].between(5170, 5172)),
       'Industry_FF'] = '[3] Oil and Petroleum Products'

df.loc[(df['WC07021'].between(2200, 2399)) |
       (df['WC07021'].between(3020, 3021)) |
       (df['WC07021'].between(3100, 3111)) |
       (df['WC07021'].between(3130, 3151)) |
       (df['WC07021'].between(3963, 3965)) |
       (df['WC07021'].between(5130, 5139)),
       'Industry_FF'] = '[4] Clothes'

df.loc[(df['WC07021'].between(2510, 2519)) |
       (df['WC07021'].between(3060, 3099)) |
       (df['WC07021'].between(3630, 3639)) |
       (df['WC07021'].between(3650, 3652)) |
       (df['WC07021'].between(3860, 3873)) |
       (df['WC07021'].between(3910, 3920)) |
       (df['WC07021'].between(3930, 3949)) |
       (df['WC07021'].between(3960, 3962)) |
       (df['WC07021'].between(5020, 5023)) |
       (df['WC07021'].between(5064, 5064)) |
       (df['WC07021'].between(5094, 5094)) |
       (df['WC07021'].between(5099, 5099)),
       'Industry_FF'] = '[5] Consumer Durables'

df.loc[(df['WC07021'].between(2800, 2829)) |
       (df['WC07021'].between(2860, 2899)) |
       (df['WC07021'].between(5160, 5169)),
       'Industry_FF'] = '[6] Chemicals'

df.loc[(df['WC07021'].between(2100, 2199)) |
       (df['WC07021'].between(2830, 2834)) |
       (df['WC07021'].between(2840, 2844)) |
       (df['WC07021'].between(5120, 5122)) |
       (df['WC07021'].between(5194, 5194)),
       'Industry_FF'] = '[7] Drugs, Soap, Perfumes, Tobacco'

df.loc[(df['WC07021'].between(800, 899)) |
       (df['WC07021'].between(1500, 1549)) |
       (df['WC07021'].between(1600, 1699)) |
       (df['WC07021'].between(1700, 1799)) |
       (df['WC07021'].between(2400, 2459)) |
       (df['WC07021'].between(2490, 2499)) |
       (df['WC07021'].between(2850, 2859)) |
       (df['WC07021'].between(2950, 2952)) |
       (df['WC07021'].between(3200, 3219)) |
       (df['WC07021'].between(3240, 3259)) |
       (df['WC07021'].between(3261, 3299)) |
       (df['WC07021'].between(3420, 3459)) |
       (df['WC07021'].between(5030, 5039)) |
       (df['WC07021'].between(5070, 5078)) |
       (df['WC07021'].between(5198, 5198)) |
       (df['WC07021'].between(5210, 5211)) |
       (df['WC07021'].between(5230, 5231)) |
       (df['WC07021'].between(5250, 5251)),
       'Industry_FF'] = '[8] Construction'

df.loc[(df['WC07021'].between(3300, 3399)),
       'Industry_FF'] = '[9] Steel'

df.loc[(df['WC07021'].between(3410, 3412)) |
       (df['WC07021'].between(3443, 3444)) |
       (df['WC07021'].between(3460, 3499)),
       'Industry_FF'] = '[10] Fabricated Products'

df.loc[(df['WC07021'].between(3510, 3629)) |
       (df['WC07021'].between(3670, 3699)) |
       (df['WC07021'].between(3810, 3839)) |
       (df['WC07021'].between(3950, 3955)) |
       (df['WC07021'].between(5060, 5063)) |
       (df['WC07021'].between(5065, 5065)) |
       (df['WC07021'].between(5080, 5081)),
       'Industry_FF'] = '[11] Machinery and Business Equipment'

df.loc[(df['WC07021'].between(3710, 3711)) |
       (df['WC07021'].between(3714, 3714)) |
       (df['WC07021'].between(3716, 3716)) |
       (df['WC07021'].between(3750, 3751)) |
       (df['WC07021'].between(3792, 3792)) |
       (df['WC07021'].between(5010, 5015)) |
       (df['WC07021'].between(5510, 5531)) |
       (df['WC07021'].between(5560, 5599)),
       'Industry_FF'] = '[12] Automobiles'

df.loc[(df['WC07021'].between(3713, 3713)) |
       (df['WC07021'].between(3715, 3715)) |
       (df['WC07021'].between(3720, 3743)) |
       (df['WC07021'].between(3760, 3769)) |
       (df['WC07021'].between(3790, 3790)) |
       (df['WC07021'].between(3795, 3795)) |
       (df['WC07021'].between(3799, 3799)) |
       (df['WC07021'].between(4000, 4013)) |
       (df['WC07021'].between(4100, 4231)) |
       (df['WC07021'].between(4400, 4789)),
       'Industry_FF'] = '[13] Transportation'

df.loc[(df['WC07021'].between(4900, 4900)) |
       (df['WC07021'].between(4910, 4911)) |
       (df['WC07021'].between(4920, 4925)) |
       (df['WC07021'].between(4930, 4932)) |
       (df['WC07021'].between(4939, 4942)),
       'Industry_FF'] = '[14] Utilities'

df.loc[(df['WC07021'].between(5260, 5271)) |
       (df['WC07021'].between(5300, 5334)) |
       (df['WC07021'].between(5390, 5499)) |
       (df['WC07021'].between(5540, 5541)) |
       (df['WC07021'].between(5550, 5551)) |
       (df['WC07021'].between(5600, 5699)) |
       (df['WC07021'].between(5700, 5750)) |
       (df['WC07021'].between(5800, 5813)) |
       (df['WC07021'].between(5890, 5890)) |
       (df['WC07021'].between(5900, 5999)),
       'Industry_FF'] = '[15] Retail'

df.loc[(df['WC07021'].between(6000, 6799)),
       'Industry_FF'] = '[16] Banks, Insurance Companies, and Other Financials'

df.loc[(df['WC07021'].between(4800, 4899)),
       'Industry_FF'] = '[17] Telecommunications'

df.loc[(df['WC07021'].between(7000, 8999)),
       'Industry_FF'] = '[18] Services'

df.loc[(df['WC07021'].between(9100, 9729)),
       'Industry_FF'] = '[19] Public Administration'

df.loc[df['Industry_FF'] == '', 'Industry_FF'] = '[20] Other'

#__________________________________________________________________________________________________________


df = df[df['Industry_FF'] != '[16] Banks, Insurance Companies, and Other Financials']
df = df[df['Industry_Div'] != '[9] Public Administration']
df = df[df['Industry_FF'] != '[14] Utilities']


## Allocation Process

#### Adding Degree Measures

In [6]:
ac = df

In [7]:

ac_final = []

for year in YEARS:

    df = pd.read_csv(f'{output_dir}{year}_investorData.csv', encoding = 'latin', index_col = False)
    df = df[df['Parent Company'].notna()]
    df['SharesHeld'] = pd.to_numeric(df['SharesHeld'])
    df = df[df.SharesHeld != 0]

    ### there are rows with shares held percentage > 1.

    ### 1. Load the Accounting Data including market caps for each ISIN;
    ac_y = ac.loc[ac['year'] == year]

    ### 2. Put market cap values on each row in Holdings Data, matching on ISIN;
    df = pd.merge(df, ac_y[['ISIN', 'MARKETCAPITALIZATION']], on = 'ISIN', how = 'left')
    df = df[df.MARKETCAPITALIZATION != 0]
    df = df[df['MARKETCAPITALIZATION'].notna()]

    ### (3. Create stock price column by dividing shares held value by number of shares;)
    df['stock_price'] = df['SharesHeldValue'] / df['SharesHeld']

    ### 4. Create holdings_percentage column by dividing shares held value by market cap
    df['Holdings_Percentage'] = df['SharesHeldValue'] / df['MARKETCAPITALIZATION']
    df['Holdings_Percentage'].fillna(0, inplace = True)

    
#   ______________________________________________________________________________________________________________________________________________
    
    
    
    ### 5. Generate different methods of measures on firm level:

    ### - Degree Centrality (weighted) based on new created holdings_percentage value
    degree = pd.read_csv(f'{adj_dir}Degree_Centrality_{year}.csv')
    degree = degree.rename(columns = {'Unnamed: 0':'investor'})
    DC = degree.set_index('investor').to_dict()['degree']
    df['degree_weighted'] = df['Holdings_Percentage'] * df['InvestorFullName'].map(DC)
    df_f = df.groupby(['ISIN'], sort = False)['degree_weighted'].sum()
    ac_y = pd.merge(ac_y, df_f, on = 'ISIN', how = 'left')
    ac_y = ac_y[ac_y['degree_weighted'].notna()]

    ### - Degree Centrality (biggest blockholder)
    df_f2 = df.loc[df.groupby(['ISIN'], sort = False)['Holdings_Percentage'].idxmax()]
    df_f2['degree_largest'] = df_f2['degree_weighted']
    ac_y = pd.merge(ac_y, df_f2[['ISIN', 'degree_largest']], on = 'ISIN', how = 'left')
    ac_y = pd.merge(ac_y, df_f2[['ISIN', 'Parent Company']], on = 'ISIN', how = 'left')
    ac_y = ac_y.rename(columns = {'Parent Company':'Biggest_Blockholder'})
    
    ### - Relative Degree Centrality (weighted)
    rdc = pd.read_csv(f'{adj_dir}Relative_Degree_Centrality_{year}.csv')
    RDC = rdc.set_index('investor_j').to_dict()['relative_degree']
    df['relative_degree_weighted'] = df['Holdings_Percentage'] * df['InvestorFullName'].map(RDC)
    df_fr = df.groupby(['ISIN'], sort = False)['relative_degree_weighted'].sum()
    ac_y = pd.merge(ac_y, df_fr, on = 'ISIN', how = 'left')

    ### - Relative Degree Centrality (biggest blockholder)
    rdcp = pd.read_csv(f'{adj_dir}Relative_Degree_Centrality_Parents_{year}.csv')
    RDCP = rdcp.set_index('investor_j').to_dict()['relative_degree']
    df['relative_degree_largest'] = df['Parent Company'].map(RDCP)
    df_fr2 = df.loc[df.groupby(['ISIN'], sort = False)['Holdings_Percentage'].idxmax()]
    df_fr2 = df_fr2.rename(columns = {'Parent Company':'Biggest_Blockholder'})
    ac_y = pd.merge(ac_y, df_fr2[['ISIN', 'relative_degree_largest']], on = 'ISIN', how = 'left')
        
    ### - BlackRock dummy; Top 3 Dummy; Stock Price
    ISIN = ac_y['ISIN'].unique()
    ac_y['BlackRock_Holdings'] = 0
    ac_y['Big_Three_Holdings'] = 0
    ac_y['Stock_Price'] = np.nan
    for i in ISIN:
        if len(df.loc[(df['ISIN'] == i) & (df['Parent Company'] == 'BlackRock')]) >= 1:
            ac_y['BlackRock_Holdings'].loc[ac_y['ISIN'] == i] = 1
        if len(df.loc[(df['ISIN'] == i) & ((df['Parent Company'] == 'BlackRock') | (df['Parent Company'] == 'Vanguard') | (df['Parent Company'] == 'State Street'))]) >= 1:
            ac_y['Big_Three_Holdings'].loc[ac_y['ISIN'] == i] = 1
        else:
            pass
        
        ac_y.loc[ac_y['ISIN'] == i, 'Stock_Price'] = df.loc[df['ISIN'] == i, 'stock_price'].mean()    

    ### - Eigenvector Centrality (weighted) based on new created holdings_percentage value
    eigen = pd.read_csv(f'{adj_dir}Eigenvector_Centrality_{year}.csv')
    eigen = eigen.rename(columns = {'Unnamed: 0':'investor'})
    EC = eigen.set_index('investor').to_dict()['eigenvector']
    df['eigenvector'] = df['Holdings_Percentage'] * df['InvestorFullName'].map(EC)
    df_e = df.groupby(['ISIN'], sort = False)['eigenvector'].sum()
    ac_y = pd.merge(ac_y, df_e, on = 'ISIN', how = 'left')
    ac_y = ac_y[ac_y['eigenvector'].notna()]
    
    ### - MSCI All Country World Index Dummy
    msci = pd.read_csv(f'{backup_dir}ACWI 2005-2021.csv')
    msci_y = msci.loc[msci['Year'] == year]
    ac_y = pd.merge(ac_y, msci_y[['ISIN', 'MSCI Dummy', 'MSCI Weighted Dummy']], on = 'ISIN', how = 'left')
    ac_y['MSCI Dummy'].fillna(0, inplace=True)
    ac_y['MSCI Weighted Dummy'].fillna(0, inplace=True)
    
    
    
    ac_final.append(ac_y)
    
    print(f'{year} finished')
    

ac_final = pd.concat(ac_final)

ac_final = ac_final[ac_final['degree_weighted'] < 1]
ac_final = ac_final[ac_final['degree_largest'] < 1]
ac_final = ac_final[ac_final['relative_degree_weighted'] < 1]
ac_final = ac_final[ac_final['relative_degree_largest'] < 1]

print('code finished')



C:\Users\DERMINA1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,13,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\DERMINA1\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2006 finished
2007 finished
2008 finished
2009 finished
2010 finished
2011 finished
2012 finished


C:\Users\DERMINA1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,13,21,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2013 finished
2014 finished
2015 finished
2016 finished
2017 finished


C:\Users\DERMINA1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2018 finished
2019 finished
2020 finished
code finished


#### Adding Control Variables

In [8]:
ac2 = ac_final

### Firm Size
ac2['log_assets'] = np.log(ac2['TOTALASSETS'])

### Return on Assets
ac2['ROA'] = ac2['NETINCOMEBASIC'] / ac2['TOTALASSETS']

### Return on Equity
ac2['ROE'] = ac2['NETINCOMEBASIC'] / ac2['TOTALEQUITY']

### Capex
ac2['CAPEX'] = ac2['CAPITALEXPENDITURES'] / ac2['TOTALASSETS']

### Log_Market Cap
ac2['log_market_cap'] = np.log(ac2['MARKETCAPITALIZATION'])

### Leverage
ac2['Leverage'] = ac2['TOTALDEBT'] / ac2['TOTALASSETS']

### Market to Book ratio
ac2['MtB-Ratio'] = ac2['MARKETCAPITALIZATION'] / (ac2['TOTALASSETS'] - ac2['TOTALLIABILITIES'])

### Operating Margin
ac2['Operating_Margin'] = ac2['OPERATINGINCOME'] / ac2['NETSALESORREVENUES']

### Tobin's Q
ac2['Tobins_Q'] = (ac2['MARKETCAPITALIZATION'] + ac2['TOTALLIABILITIES']) / ((ac2['TOTALASSETS'] - ac2['TOTALLIABILITIES']) + ac2['TOTALLIABILITIES'])

### Research & Development
ac2['R&D'] = ac2['RESEARCHDEVELOPMENT'] / ac2['TOTALASSETS']

### Intangibles
ac2['Intangibles'] = ac2['TOTALINTANGIBLEOTASSETSNET'] / ac2['TOTALASSETS']

### Price to Earnings Ratio
ac2['PtE-Ratio'] = ac2['MARKETCAPITALIZATION'] / ac2['NETINCOMEBASIC']


### Sales Growth
ac_old = []
ac00 = ac[ac['year'] == 2000]
ac01 = ac[ac['year'] == 2001]
ac02 = ac[ac['year'] == 2002]
ac03 = ac[ac['year'] == 2003]
ac04 = ac[ac['year'] == 2004]
ac05 = ac[ac['year'] == 2005]
ac06 = ac[ac['year'] == 2006]
ac_old.append(ac00)
ac_old.append(ac01)
ac_old.append(ac02)
ac_old.append(ac03)
ac_old.append(ac04)
ac_old.append(ac05)
ac_old.append(ac06)
ac_old = pd.concat(ac_old)
ac2 = ac2.append(ac_old).drop_duplicates(['ISIN','year'], keep='first')


AC_FINAL = ac2[['ISIN', 'NAME', 'year', 'WC07021', 'Industry_Div', 'Industry_FF', 'GEOGN',
                
                'CAPITALEXPENDITURES', 'MARKETCAPITALIZATION', 'NETINCOMEBASIC', 'NETSALESORREVENUES', 'OPERATINGINCOME',
                'TOTALASSETS', 'TOTALDEBT', 'TOTALLIABILITIES', 'TOTALINTANGIBLEOTASSETSNET', 'RESEARCHDEVELOPMENT', 'CURRENTASSETSTOTAL',
                'PROPERTYPLANTEQUIPNET', 'SELLINGGENERALADMINISTRAT',
                
                'TOTALEQUITY', 'R&D', 'Intangibles', 'Operating_Margin', 'Stock_Price', 'log_assets', 'ROA', 'ROE', 'CAPEX', 'log_market_cap',
                'Leverage', 'MtB-Ratio', 'PtE-Ratio', 'Tobins_Q',
                
                'Biggest_Blockholder', 'BlackRock_Holdings', 'Big_Three_Holdings',
                'degree_weighted', 'degree_largest', 'relative_degree_weighted', 'relative_degree_largest', 'eigenvector', 'MSCI Dummy', 'MSCI Weighted Dummy']]


AC_FINAL.replace([np.inf, -np.inf], 0, inplace=True)

AC_FINAL.sort_values(by=['NAME', 'year'], inplace = True)
AC_FINAL.reset_index(drop=True, inplace=True)
# AC_FINAL.to_csv(f'{acc_dir}Accounting_Data_wdead.csv')
# AC_FINAL.to_excel(f'{acc_dir}Accounting_Data_wdead.xlsx')
# AC_FINAL2 = AC_FINAL[~AC_FINAL['NAME'].str.contains(' DEAD ')]
AC_FINAL.to_csv(f'{acc_dir}Accounting_Data.csv')
AC_FINAL.to_excel(f'{acc_dir}Accounting_Data.xlsx')





C:\Users\DERMINA1\Anaconda3\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\Users\DERMINA1\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
ac2.info(verbose = True, null_counts = True)

C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_35504/4005317468.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  ac2.info(verbose = True, null_counts = True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 33292 entries, 0 to 158181
Data columns (total 229 columns):
 #    Column                         Non-Null Count  Dtype   
---   ------                         --------------  -----   
 0    DSCD                           33292 non-null  object  
 1    ISIN                           33292 non-null  object  
 2    WC06008                        33292 non-null  object  
 3    MNEM                           33292 non-null  object  
 4    NAME                           33292 non-null  object  
 5    WC06001                        33292 non-null  object  
 6    WC06091                        33292 non-null  object  
 7    WC07021                        33292 non-null  int32   
 8    WC07022                        33292 non-null  object  
 9    WC07023                        33292 non-null  object  
 10   WC07024                        33292 non-null  object  
 11   WC07025                        33292 non-null  object  
 12   WC07026        

In [10]:
FU = AC_FINAL[['ISIN', 'year']]

FU.to_csv(f'{backup_dir}Unique Firm Year Observations.csv')

In [11]:
### DEAD companies after delistings???

# acf = pd.read_csv(f'{acc_dir}Accounting_Data.csv')
# acfd = acf[acf['NAME'].str.contains(' DEAD ')]
# acfd[['ISIN', 'NAME', 'year', 'GEOGN', 'NETINCOMEBASIC', 'TOTALASSETS', 'ROA']][acfd['ROA'] <= -2]

# acfd['delist_date'] = acfd['NAME'].str[-8:]

# acfd['delist_date'] = pd.to_datetime(acfd['delist_date'], errors = 'coerce')
# acfd['delist_year'] = acfd['delist_date'].dt.year

# acfd[['ISIN', 'NAME', 'year', 'GEOGN', 'NETINCOMEBASIC', 'TOTALASSETS', 'ROA']][acfd['delist_year'] <= acfd['year']].tail(60)